# このドキュメントは？

論文「A Contextual-Bandit Approach to　Personalized News Article Recommendation」を読み、その概要のメモするためのもの。

この論文は、ニュースサイトの記事レコメンドをBanditアルゴリズムのアプローチで解決し、非常に多くの引用を集めた。


### bib

``` 
@inproceedings{li2010contextual,
  title={A contextual-bandit approach to personalized news article recommendation},
  author={Li, Lihong and Chu, Wei and Langford, John and Schapire, Robert E},
  booktitle={Proceedings of the 19th international conference on World wide web},
  pages={661--670},
  year={2010},
  organization={ACM}
}
```

## ABSTRACT

パス

## 1. INTRODUCTION

パス

## 2. FORMULATION & RELATED WORK


### 2.1 A Multi-armed Bandit Formulation

ニュースサイトのようなcontextの情報を扱うバンディットアルゴリズムを、私たちは contextual-bandit algorithm と呼ぶ。

例えば、アルゴリズムAでは、試行(tial) t=1,2,3,... と進み、それぞれにtにおいて以下が実行される。

 1. ユーザー$u_t$とアーム（表示する記事）セット$A_t$があり、それぞれの持つ特徴ベクトルが$x_{t,a}\;(a \in A_t)$とする。これはcontextを表している。
 
 2. 前回の試行($t-1$)の観測に基づき、アーム$a_t \in A_t$を選択し、報酬$r_{t,a_t}$が得られる。この報酬はユーザーとアームだけに依存すると仮定する。
 
 3. アームを選択する戦術を改善するために、新しい観測値セット($x_{t,a}, a_t, r_{r,a_t}$)を得る。重要なことは、選択されなかったアームの報酬$r_{t,a}$がフィードバックされないことだ。
 
 上記の処理において、試行回数$T$の合計報酬は $\sum_{t=1}^{T} r_{t,a_t}$ となる。
 
 同様に、最適な合計報酬の予測を $E[\sum_{t=1}^T r_{t,a_t^*}]$ と定義する。　$a_t^*$ とは報酬が最大と推定されたアームである。
 
 この報酬合計を最大化するようにアルゴリズムを設計することが私たちの目標です。
 つまり、　regretを最小化するようにアーム選択を予測します。
ここでは 試行回数Tにおけるregret $R_A(T)$ を以下で公式化します。

$$
R_A(T) \overset{def}{=} E[\sum_{t=1}^{T} r_{t,a_t^*}] - E[\sum_{t=1}^{T} r_{t,a_t}] \;\; ...(1)
$$

特殊な contextual bandit 問題のケースでは、(i) アームセット $A_t$ は一定でなければなりません、(ii)ユーザー$u_t$ (context($x_{t,1},...,x_{t,K}$)と等価)はあらゆるtで一定です。これはバンディットアルゴリズムと同じであることから、context-free banditと呼びます。

一方で、記事レコメンデーションでは、アームとしてプールされた記事を閲覧します。
表示した記事がクリックされると、報酬として１が与えられ、そうでなければ０です。
この論文では、報酬の予測はクリック率(CTR, Click Throug Rate)となります。
そして、このCTRが最大になるように記事を選択することで、合計のクリック数の最大を目指します。

幸運なことに、ニュースサイトを運営することで様々な情報を活用できます。
例えば、定年後の計画よりもiPod製品に興味があるかなどです。
これらの情報をコンパクトにサマライズすることで、バンディットアルゴリズムがCTR予測をできるartcle/user情報を生成しました。
これによって、新しい記事やユーザーが現れても、記事をレコメンドできます。


### 2.2 Existing Bandit Algorithms

UCB(UppeBound) アルゴリズムは、シンプルな$\epsilon$ -greedyアルゴリズムに対して信頼区間の上側を探索優先度に採用した手法です。
ある試行回数時点tで、アクションaの報酬の期待値を$\hat{\mu}_{t,a}$と推定し、同時に真の報酬との偏差（信頼区間）を$c_{t,a} > |\hat{\mu}_{t,a} - \mu_{t,a}|$ を算出します。
この信頼区間の上側を評価し、$arg \; max_a (\hat{\mu}_{t,a} + c_{t,a})$に基づいてアームを選択します。


## 3. ALGORITHM

私たちはUCBをベースに、報酬を線形モデルで表現し、信頼区間を閉空間で効果的に計算できるLinUCBと呼ぶアルゴリズムを提案します。
はじめにシンプルなモデルを3.1節で紹介し、より一般化した複合モデルを3.2節で紹介します。
ただし、LinUCBはニュースのパーソナライズレコメンデーションに適したバンディットアルゴリズムであることに言及しておきます。

### 3.1 LinUCB with Disjoint Linear Models

d-次元の変数ベクトル$\mathbf{x}_{t,a}$とそれに対応する係数ベクトル$\theta_a^*$を使い、その時点tでのアームaの期待報酬を以下のように定式化します。

$$
E[r_{t,a}|\mathbf{x}_{t,a}] = \mathbf{x}_{t,a}^{T} \theta_a^* \;\;...(2)
$$

このモデルは disjoint であると言えます、なぜならパラメータはアーム（ニュース記事）同士で共有されていないからです。
ここで行列$D_a$を定義します。
この次元は$(m \times d)$で、$m$は学習データのインスタンス数であり、そのアームで過去に観測されたコンテキストの数です。
そして、その過去のコンテキストに対応する応答ベクトルを$b_a\in R^m$とします。(つまり、click/no-clickのユーザーフィードバックです)
そのアーム（記事）aの過去の表示結果である$(D_a, c_a)$に対して、ridge回帰を適応することで、以下のように係数を推定します。

$$
\hat{\theta}_a = (D_a^T D_a + I_d)^{-1} D_t^T c_a \;\;...(3)
$$

$I_d$は$d \times d$の単位行列です。
仮に$c_a$の各要素が互いに独立であるならば、確率$(1- \delta)$で推定誤差は以下の範囲に収まります。

$$
|\mathbf{x}_{t,a}^T \hat{\theta}_a - E[r_{t,a}|\mathbf{x}_{t,a}]| \leq \alpha \sqrt{\mathbf{x}_{t,a}^T(D_a^T D_a + I_d)^{-1} \mathbf{x}_{t,a}} \;\; ...(4)
$$

$$
\theta > 0 \;\; ; \;\; \mathbf{x}_{t,a}^T \in R^d \;\; ; \;\; \alpha = 1+\sqrt{ln(1/\delta)/2}
$$

よって、報酬を線形回帰モデルで予測した場合のUCB型のアーム選択の戦術は、最終的には下記になります。

$$
a_t \overset{def}{=} arg \; \max_{a \in A_t} (x_{t,a}^T \hat{\theta}_a + \alpha \sqrt{x_{t,a}^T A_a^{-1}x_{t,a}}) \;\; ...(5)
$$

$$
where \;\; A_a \overset{def}{=} D_a^T D_a + I_d
$$

信頼区間である(4)は、意図的に(?may be motivated?) 他の原理から派生している。

ridge回帰はベイズ推定から考えると、係数の事後分布を$p(\theta_a)$とし、平均が$\hat{\theta}_a$で共分散が$A_a^{-1}$のガウス分布と考えられる。
このモデルでは、推定報酬の$x_{t,a}^T \theta_a^*$の分散は$x_{t,a}^T A_a^{-1} x_{t,a}$であり、その標準偏差は$\sqrt{x_{t,a}^T A_a^{-1} x_{t,a}}$である。

(中略)

最後に、私たちは入力される変数ベクトル$x_{t,a}$が、ある正規分布から抽出され、独立であることを仮定した。（式(2)における仮定）
他の研究でも似たアプローチをとったものはある。例えばPavlidis et al.などは最小二乗誤差の最適化で係数を求めるアプローチをとっている。

次の3.2節では、このアルゴリズムを拡張したもの紹介する。
この拡張は、Pavlidis et al.の研究ではカバーされていないものだ。

![algorithm1](img/algorithm1.png)


### 3.1 LinUCB with Hybrid Linear Models

Algorithm 1 では$D_a^T D_a$の逆行列を計算した。
$D_a$は行が学習データの変数に対応している。
この行列は$d \times d$の固定的な次元で、増加的に更新されていく。
さらにこれらの逆行列は簡単にパラメータとして計算され、式(3)の$\hat{\theta}_a$が求められる。
これらはdisjintなので、別々に計算される。
この章では、hybridモデルを用いたケースを紹介する。

多くアプリケーションでは、全てのアームで変数を共通して使うと都合が良い。
例えば、新しいニュース記事のレコメンデーションを行うなどだ。
この変数の共通化を実現するために、式(2)の右辺を以下のように変更した。

$$
E[r_{t,a}|x_{t,a}] = z_{t,a}^T \beta ^ * + x_{t,a}^T \theta_a^*
$$

ここで、$z_{t,a} \in R^k $ は現在のユーザー/記事の変数セットで、$\beta ^ *$は未知のパラメータ係数で、これは全てのアームが共通して持つ。
つまり、アームは共通の$\beta ^ *$と個別の$\theta_a^*$の二つの係数を持つことになる。

この導入により、Algorithm 1 のような独立したパラメータの計算はできなくなる。
幸いにも、UCBを計算する効果的な方法がある。
この導出は、block matrix inversion techniques に強く依存する。
full-paperではその詳細を記載している。
この Algorithm 2　のpseudocodeのみを記載する。
5から12行がridge回帰による係数導出で、13行目が係数を再帰的に求めている。

![algorithm2](img/algorithm2.png)

このアルゴリズムの重要な箇所は、blocksを構築していることで計算を効果的にしている点だ。
$(A_0, b_0, A_a, B_a, b_a)$ は全て固定的な次元で、増加的に更新される。
さらに、$A_t$にもはや存在しないアーム（記事）については、計算もされない。
最後に、逆行列（$A_0^{-1}$, $A_a^{-1}$）を計算しキャッシュ化することで、計算の複雑さを抑制していることだ。


## 4. EVALUATION METHODOLOGY

Banditアルゴリズムの評価は、本質的にはオンラインで行うべきだが、その実施は難しい。
そのため、オフラインdataを使った評価を行うのだが、これはすでに観測済みの範囲でしか評価できないデメリットがある。
この問題は強化学習における''off-policy evaluation problem''と呼ばれ、[23]の研究を参照してほしい。

一つの解決法として、観測済みのオフラインデータからシミュレーターを構築して、そのシミュレーターが提供する環境下での性能を検証する。
しかし、これもシミュレーターの設計に依存する点が問題になる。
私たちは他のよりシンプルで過去ログを使った解決法を提案する。

(中略)

ここに未知の分布 $D$　を過程し、そこから観測データ（tuples)がi.i.d.で取得すると過程します。
その観測データは、$(x_1, ..., x_K, r_1, ..., r_K)$ の形式をとり、それぞれは全てのアームの観測された変数と潜在した報酬です。
また、大きなサイズのログイベントにアクセスすることを仮定し、それはこの世界でのポリシーの結果であるとします。
それぞれのイベントはcontextベクトル$x_1, ..., x_K$で構成され、選択されたアーム$a$によって報酬$r_a$が観測されます。
重大なことは、選択されたただ一つのアームに対応する$r_a$は、一様乱数で取得されたアームであることです。
単純に説明すれば、私たちは観測済みのログイベントを無限の時間をかけて取得したものと見なし、一方で、私たちは明確なイベントの実際の上限数を設定して、手法を評価します。

私たちの目的は、このオフラインデータを使って、バンディットアルゴリズム$\pi$を評価することです。
公式としては、アルゴリズムは時点tに置いてアーム$a_t$を選択するために、（おそらく無作為に）写像されたもので、それまでのイベント履歴$h_{t-1}$と現在のcontextベクトル$x_{t1},...,x_{tK}$に基づいて動作します。

私たちの提案する評価方法を Algorithm 3 で示します。

![algorithm3](img/algorithm3.png)

この手法では、ポリシー$\pi$(つまりアルゴリズム）と目標の''good''イベントの数$T$を入力します。
私たちはログイベントのストリームを一つずつ順番に流していきます。
時点tでの履歴$h_{t-1}$が与えられた時、ポリシー$\pi$が観測したデータと同じアーム$a$を選択した場合は、そのイベントを保存し、履歴に加え、そして合計の報酬$R_t$に加えます。
一方で、もしポリシー$pi$が観測されたデータと異なるアームを選択した場合は、このイベント自体を無視して、更新は行わず、次のイベント処理に移ります。

注意すべきは、過去に用いたポリシー(logging policy)がアームを一様乱数で選択するものであり、ログイベントはそれによって観測されたものであることです。
それによって、それぞれのイベントは全てに対して独立が保証されています。
つまり、分布Dから選択されたイベントであるということです。
その結果、私たちは二つの処理が等価であることを証明することができます。
一つめは分布DからのT回の実測世界(real-world）イベントについての評価。
二つめは、それによって観測されたログを提案する手法でのストリーミング評価。

これを定理として表現すると、以下のようになります。

![theorem](img/theorem.png)


(以下は上記の定理の証明。省略。)

## 5. EXPERIMENT

上記で提案したLinUCBを、Yahoo!のデータに対して、提案したオフライン評価方法で評価しました。

### 5.1 Yahoo! Today Module

検証に用いるYahoo!　のニュース表示は以下のようなものです。

![figure1](img/figure1.png)


### 5.2 Experiment Setup

このサブセクションでは、検証におけるデータの処理方法などの事前設定について紹介します。

#### 5.2.1 Data Collection

2009年の5月から、ランダムでデータを収集しました。
ある一定の閲覧数のあるユーザーがある確率で選択され、彼女をランダムバケットに入れます。
このランダムバケットに入ったユーザーには、ランダムで記事を表示します。
バイアスを抑制するために、評価ではF1に表示された記事しか用いません。

5月1日にニュースを見た470万人のユーザーをランダムバケットに入れ、この1日のデータ(これをtunning dataと呼びます)を学習データとして、バンディットアルゴリズムのパラメータとします。
そして、その学習結果に対して、次の一週間のデータ（これをevaluation dataと呼びます）をテストにして、評価を行います。

#### 5.2.2  Feature Construction

ここでは、ユーザーと記事の変数の生成方法について説明します。
disjointモデル(algorithm1)とhybridモデル(algorithm2)の二種類の変数セットがあります。

ユーザーの変数をそのまま利用することからはじめていきましょう。
変数はsupport(その変数を持っているユーザーの割合)が0.1よりも高いものだけを使います。
これにより、ユーザーは約1000個のカテゴリカル変数で表現可能になります。
具体的な変数として: (i)デモグラフィック; (ii) 住所情報; (iii) 行動情報;　があります。
(iii)はYahoo! の各プロパティの閲覧行動を変数化したものです。

同様に、記事の変数についても説明します。
(i)URLカテゴリ; (ii)編集カテゴリ; で、約100個のカテゴリカル変数です。
(ii)は記事の編集者が設定するサマライズタグです。

これらの変数をバイナリ化して、変数を標準化します。
この処理については、[12]の研究を参考にしました。
これらによって、記事は83、ユーザーは1193のエンティティで表現されました。

また、変数の次元圧縮のために、[13]の研究を参考にして、2008年9月のユーザーのニュースカテゴリの閲覧行動からユーザー同士の共通行動を解析してクラスタ化しました。
より詳細については下記になります。

![detail1](img/detail1.png)

上記の処理により、施行t時点で選択された記事aは6次元の変数ベクトル$x_{t,a}$で表現され、ユーザーも同様の6次元の変数ベクトル$u_t$で表現されます。
この場合、記事の変数は共有化されていないので、これは3章で定義したdisjointの線形回帰モデルだと言えます。

この$x_{t,a}$と$u_t$の外積によって$6 \times 6 =36$の値を取得できます。これをhybridモデル(式(6))で利用する記事で共有される変数ベクトル$z_{t,a}$とします。
よって$(z_{t,a}, x_{t,a})$はhybridの線形モデルで利用されます。
$z_{t,a}$はユーザーと記事の相互作用変数で, $x_{t,a}$はユーザーのみの変数でもあります。

私たちはユーザーと記事を5つの共通クラスタ変数と1つの切片の6次元に変換しました。
この数については先行研究[13]を参考にしたものですが、同時にwebの掲載を行う上で現実的な処理速度を担保するためでもあります。


### 5.3 Compaired Algorithms

対象実験に用いるアルゴリズムを紹介します。

##### Ⅰ. 変数を取らないアルゴリズム

context free 型のアルゴリズムです。

1. random
2. $\epsilon - greedy$
3. ucb
4. omniscient

##### Ⅱ. 'warm start' のあるアルゴリズム

パーソナライゼーションへの前提段階のアルゴリズムで、ユーザーと個別記事のCTRを事前に学習してからcontext free型のアルゴリズムを実行する、というアイデアです。
先行研究[12]に基づいたロジスティック回帰モデルを用いました。

1. $\epsilon - greedy(warm)$
2. $ucb(warm)$

##### Ⅲ. Algorithms that learn user-specific CTRs online

1. $\epsilon - greedy(seg)$ : ユーザーを単純に5つのsegmentに分けて処理します(seg)。
2. $ucb(seg)$
3. $\epsilon - greedy(disjoint)$
4. $\epsilon - greedy(hybrid)$
5. $linucb(hybrid)$


### 5.4 Performance Metric

これから、各手法のCTRを比較するが、ランダム選出手法のCTRの何倍のCTRになったかを報告する。
以降でCTRと記載があれば、ランダムとの相対的なCTR比率であることに注意。

Yahoo! へのトラフィックを、[3]の研究を参考にして、２つのバケットに分ける。
一つは''learning bucket''である。一般的にはトラフィックのうち、ごく少数が割り振られ、各種のバンディットアルゴリムが学習するためのデータとして使われる。
もう一つは''deployment bucket''で、これは学習されたCTRを用いてYahoo!がユーザーにコンテンtぬを表示する通常のFrontPageである。
learning bucketでのデータが更新されれば、各手法は学習され記事の予測CTRが更新される。
いずれのbucketでの性能も、algorithm3の手法で評価した。

### 5.5 Experimental Results

#### 5.5.1 Results for Tuning Data

ucbの場合は$\alpha$, $\epsilon$-greedyの場合は$\epsilon$のチューニングが必要になる。
tunning data　を用いて決定した結果は以下の通りです。

![figure2](img/figure2.png)

以下が、各手法のCTRを比較した表である。

![table1](img/table1.png)

deployment bucket のデータ量は大きいため、そこで高いCTRを出すことが重要だが、learning bucketでの高いCTRは学習効率が良いことを示している。両方とも重要な指標だ。

上の図の％は、$\epsilon$-greedyをベースラインにしたCTRの改善度(lift)である。
size=は、パラメータチューニングに用いたデータ量を表している。
sizeが少ないほどスパースな状況での性能を表している。


#### 5.5.2 Results for Evaluation Data

##### On the Use of Features

ベースラインに比べて10%近いCTR改善が見られるため、提案手法は有効なニュース記事のパーソナルレコメンデーションである。

より詳細なCTR改善を可視化したものが、下記のFigure3である。
このグラフはベースモデルのCTR(base ctr)からの改善CTR(lifted ctr)を比較したもので、赤い十字記号がベースである$\epsilon$-greedyであり、青い丸点が比較する手法である。
それぞれ、最も多く閲覧された50個の記事ごとに評価CTRをプロットしている。

提案手法であるlinucbは、ユーザーの情報を学習しない手法に比べて、記事ごとに改善が確認できる。

![figure3](img/figure3.png)


##### On the Size of Data

記事の数は非常に多いため、CTR評価を行う deployment backet における表示回数は少ない方が良い。

Table1　でも、size=[30,20,10,5,1%]ごとにデータサイズでの性能を比較し、deployment backetでのCTRの高さを評価している。

より良い可視化として、Figure4を作成した。
データのスパースレベルごとの、各アルゴリズムの性能を比較したものだ。
どのレベルでも各手法は機能している。
スパースレベルが1%出会っても、linucbがucbを上回っている。

また、ucbが$\epsilon$-greedyよりも、スパース状況におけるdeployment backetでのCTR(=学習速度)に優れていた。この傾向はよりスパースな状況で、さらに顕著になる。

さらに、ucb(seg), linucb(disjoint), linucb(hybrid)の性能の違いがスパースになるにつれ顕著になっている。
linucb(hybrid)がデータの少ない状態で高い性能を発揮できているのは、記事の共通の変数をもち、学習結果を転用しているためだと考えられる。


![figure4](img/figure4.png)


![figure5](img/figure5.png)



## 6. CONCLUSIONS

pass

